In [1]:
from op_analytics.cli.subcommands.pulls.growthepie.dataaccess import GrowThePie
from op_analytics.coreutils.duckdb_inmem.client import init_client

duckdb_ctx = init_client()

2025-01-03 11:35:56 [info     ] initialized duckdb at /var/folders/tz/g2vwn0qx5ll32j7jfsshhps40000gn/T/5hqi0aak/op-analytics.duck.db filename=client.py lineno=61 process=32102
2025-01-03 11:35:56 [info     ] loaded vault from .env file    filename=vault.py lineno=32 process=32102
2025-01-03 11:35:56 [debug    ] loaded vault: 18 items         filename=vault.py lineno=76 process=32102


In [7]:
from op_analytics.coreutils.clickhouse.oplabs import generate_clickhouse_ddl

fundamentals = GrowThePie.FUNDAMENTALS_SUMMARY.read(min_date="2024-11-01")

print(duckdb_ctx.client.sql(f"DESCRIBE {fundamentals}"))

print(duckdb_ctx.client.sql(f"SELECT * FROM {fundamentals} LIMIT 10"))


# One time to help generate the DDL for the destination table.
ddl = generate_clickhouse_ddl(
    relation=duckdb_ctx.client.sql(f"SELECT * FROM {fundamentals} LIMIT 10"),
    table_name=GrowThePie.FUNDAMENTALS_SUMMARY.value,
)
print(ddl)

2025-01-03 11:39:54 [info     ] querying markers for 'growthepie/chains_daily_fundamentals_v1' DateFilter(min_date=datetime.date(2024, 11, 1), max_date=None, datevals=None) filename=dailydata.py lineno=119 process=32102
2025-01-03 11:39:54 [info     ] 65 markers found               filename=dailydata.py lineno=133 max_dt=2025-01-01 min_dt=2024-11-01 process=32102
2025-01-03 11:39:54 [info     ] 62 distinct paths              filename=dailydata.py lineno=143 process=32102
2025-01-03 11:39:56 [info     ] registered view: 'growthepie_chains_daily_fundamentals_v1' using 62 parquet paths filename=client.py lineno=170 process=32102
┌─────────────────────────────────────────┐
│                  name                   │
│                 varchar                 │
├─────────────────────────────────────────┤
│ growthepie_chains_daily_fundamentals_v1 │
└─────────────────────────────────────────┘

┌─────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name │ column_type │  

In [8]:
metadata = GrowThePie.CHAIN_METADATA.read(min_date="2024-11-01")

print(duckdb_ctx.client.sql(f"DESCRIBE {metadata}"))

print(duckdb_ctx.client.sql(f"SELECT * FROM {metadata} LIMIT 10"))


# One time to help generate the DDL for the destination table.
ddl = generate_clickhouse_ddl(
    relation=duckdb_ctx.client.sql(f"SELECT * FROM {metadata} LIMIT 10"),
    table_name=GrowThePie.CHAIN_METADATA.value,
)
print(ddl)

2025-01-03 11:40:28 [info     ] querying markers for 'growthepie/chains_metadata_v1' DateFilter(min_date=datetime.date(2024, 11, 1), max_date=None, datevals=None) filename=dailydata.py lineno=119 process=32102
2025-01-03 11:40:29 [info     ] 2 markers found                filename=dailydata.py lineno=133 max_dt=2025-01-03 min_dt=2025-01-03 process=32102
2025-01-03 11:40:29 [info     ] 1 distinct paths               filename=dailydata.py lineno=143 process=32102
2025-01-03 11:40:31 [info     ] registered view: 'growthepie_chains_metadata_v1' using 1 parquet paths filename=client.py lineno=170 process=32102
┌─────────────────────────────────────────┐
│                  name                   │
│                 varchar                 │
├─────────────────────────────────────────┤
│ growthepie_chains_daily_fundamentals_v1 │
│ growthepie_chains_metadata_v1           │
└─────────────────────────────────────────┘

┌─────────────────┬───────────────────────────────────────────────────────────

KeyError: 'VARCHAR[]'

┌────────────────┬────────────┬────────────────────┬────────────┐
│   metric_key   │ origin_key │       value        │     dt     │
│    varchar     │  varchar   │       double       │    date    │
├────────────────┼────────────┼────────────────────┼────────────┤
│ daa            │ starknet   │             7245.0 │ 2024-11-01 │
│ txcount        │ rhino      │             4426.0 │ 2024-11-01 │
│ daa            │ rhino      │             1641.0 │ 2024-11-01 │
│ txcount        │ starknet   │            59311.0 │ 2024-11-01 │
│ txcount        │ blast      │           513018.0 │ 2024-11-01 │
│ daa            │ blast      │            19741.0 │ 2024-11-01 │
│ aa_last7d      │ orderly    │               55.0 │ 2024-11-01 │
│ txcount        │ worldchain │           520595.0 │ 2024-11-01 │
│ gas_per_second │ blast      │ 2.4999156911805556 │ 2024-11-01 │
│ txcount        │ gravity    │          2003045.0 │ 2024-11-01 │
├────────────────┴────────────┴────────────────────┴────────────┤
│ 10 rows 

CREATE TABLE IF NOT EXISTS chains_daily_fundamentals_v1
(
    `metric_key` String,
    `origin_key` String,
    `value` Float64,
    `dt` Date
)
ENGINE = ReplacingMergeTree



In [ ]:
ddl = generate_clickhouse_ddl(
    relation=duckdb_ctx.client.sql(f"SELECT * FROM {fundamentals} LIMIT 10"),
    table_name=GrowThePie.CHAIN_METADATA.value,
)

print(ddl)

In [6]:
from op_analytics.coreutils.partitioned.dailydata import insert_daily_data_to_clickhouse


insert_daily_data_to_clickhouse(
    GrowThePie.FUNDAMENTALS_SUMMARY,
    min_date="2021-06-01",
    max_date=None
)

2025-01-03 11:36:09 [info     ] incremental load to clickhouse filename=dailydata.py lineno=200 max_date=None min_date=2021-06-01 process=32102 used_clickhouse_watermark=False
2025-01-03 11:36:09 [info     ] querying markers for 'growthepie/chains_daily_fundamentals_v1' DateFilter(min_date=datetime.date(2021, 6, 1), max_date=None, datevals=None) filename=dailydata.py lineno=119 process=32102
2025-01-03 11:36:10 [info     ] 1314 markers found             filename=dailydata.py lineno=133 max_dt=2025-01-01 min_dt=2021-06-01 process=32102
2025-01-03 11:36:10 [info     ] 1311 distinct paths            filename=dailydata.py lineno=143 process=32102
2025-01-03 11:36:12 [info     ] registered view: 'growthepie_chains_daily_fundamentals_v1' using 1311 parquet paths filename=client.py lineno=170 process=32102
┌─────────────────────────────────────────┐
│                  name                   │
│                 varchar                 │
├─────────────────────────────────────────┤
│ growthepie_